Please don’t worry about spending hours making it very tidy and well commented as long as the input data is one of the following:

- The original data files Leanne provided us with
- The csv I created with co-ordinates of each station
- Any publicly available data (but just note the link rather than attaching the csv)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp

import scipy.stats as sps

In [ ]:
#Input data
df_leavers = pd.read_csv('LAS_Leavers_070319.csv') #original leavers data from LAS
postcode = pd.read_csv('org_L8_postcodes.csv') #postcode data
latlong = pd.read_csv('station_pc_latlong.csv') #station coord data from Liam on L7 level
staff_in_tfl = pd.read_csv('staffFinal_v4.csv') #current staff data with their travel duration from Rachel
df_leavers_1 = pd.read_csv("LEAVERS_WITH_STATION3.csv") #leavers data with coord data from Liam on L8 level, but duplicated records

In [ ]:
##A. Correlation with travel duration

#Merge postcode and L8 data
df_leavers_1_SEC = df_leavers_1[(df_leavers_1['Org L5'] == '308 SEC Sector Operations L5')]
df_leavers_1_SEC = df_leavers_1_SEC.drop_duplicates()
SEC_postcodes = df_leavers_1_SEC[['Org L8','WorkLocation','PostcodeWork']].drop_duplicates()

In [ ]:
SEC_postcodes.columns = ['Org L8', 'WorkLocation','Postcode']
SEC_postcodes = pd.merge(SEC_postcodes, latlong, on='Postcode').drop_duplicates()
SEC_postcodes = SEC_postcodes[['Org L8', 'Latitude', 'Longitude']]
SEC_postcodes = SEC_postcodes.groupby('Org L8').mean() #Take the centroid of different stations within the same L8 Department
SEC_postcodes.head()

In [ ]:
#Put the centroid LatLong to the original leavers data
leavers_SEC_loc = pd.merge(df_leavers, SEC_postcodes, on='Org L8').drop_duplicates()
leavers_SEC_loc.head()

In [ ]:
#Get the mean travel duration of current staff per each L8 department from Rachel's data
staff_in_tfl = staff_in_tfl.dropna()
staff_in_tfl_SEC = staff_in_tfl[(staff_in_tfl['Directorate'] == '308 SEC Sector Operations L5')]
mean = staff_in_tfl_SEC.groupby(['Department'],as_index=False).agg({"travelDuration": "mean"})
mean.columns = ('Org L8','meanTravelDuration')

In [ ]:
#Get the turnover rate per each L8 department in 2018
#Turnover rate = number of leavers in 2018/ number of current staff from current staff data

#Get the date format for each records
leavers_SEC_loc['Start Data'] = pd.to_datetime(leavers_SEC_loc['Employee Latest Start Date'])
leavers_SEC_loc['Start Month'] = leavers_SEC_loc['Start Data'].map(lambda x: x.month)
leavers_SEC_loc['Start Year'] = leavers_SEC_loc['Start Data'].map(lambda x: x.year)

leavers_SEC_loc['Termination Date'] = pd.to_datetime(leavers_SEC_loc['Termination Date'])
leavers_SEC_loc['Termination Month'] = leavers_SEC_loc['Termination Date'].map(lambda x: x.month)
leavers_SEC_loc['Termination Year'] = leavers_SEC_loc['Termination Date'].map(lambda x: x.year)

leavers_SEC_loc['Notified Date'] = pd.to_datetime(leavers_SEC_loc['Notified Date'])
leavers_SEC_loc['Notified Month'] = leavers_SEC_loc['Notified Date'].map(lambda x: x.month)
leavers_SEC_loc['Notified Year'] = leavers_SEC_loc['Notified Date'].map(lambda x: x.year)

In [ ]:
#Get number of leavers in 2018
leavers_SEC_count_2018 = leavers_SEC_loc[leavers_SEC_loc['Termination Year'] == 2018]
leavers_SEC_count_2018 = leavers_SEC_count_2018[['Org L8', 'Longitude', 'Latitude']]
leavers_SEC_count_2018['leavers_count'] = (leavers_SEC_count_2018.groupby(['Longitude', 'Latitude'])['Org L8'].transform('count'))
leavers_SEC_count_2018 = leavers_SEC_count_2018.drop_duplicates()

leavers_SEC_count_2018.head()

In [ ]:
#Get number of current staff
staff_in_sec = staff_in_tfl_SEC.groupby(['Department'],as_index=False).agg({"IDnumber": "count"})
staff_in_sec.columns = ('Org L8','InStaff')

In [ ]:
#Get turnover rate
turnover = pd.merge(leavers_SEC_count_2018, staff_in_sec, on = 'Org L8')
turnover['rates'] = turnover['leavers_count']/turnover['InStaff']
turnover.head()

In [ ]:
#Merge table of turnover rate with mean travel duration
turnover = pd.merge(turnover, mean, on = 'Org L8')

In [ ]:
turnover.head()

In [ ]:
#Creating scatter plot of turnover rate and mean travel duration
x = turnover['meanTravelDuration']
y = turnover['rates']

# This line creates the figure. 
plt.figure(figsize=(7,7))
plt.xlabel("Mean Travel Duration", fontsize="16")
plt.ylabel("Turnover Rates", fontsize="16")
plt.plot(x,y,'b.')
plt.tick_params(labelsize = "14")
plt.title("Turnover Rates of SEC department\n & Mean Travel Duration", fontsize="20")

In [ ]:
#Get correlation value of turnover rates and mean travel duration
turnover['meanTravelDuration'].corr(turnover['rates'])

In [ ]:
##B. Correlation with workload

#Load incidents data
incident1 = pd.read_csv('Incidents/incidentsQ1.csv')
incident2 = pd.read_csv('Incidents/incidentsQ2.csv')
incident3 = pd.read_csv('Incidents/incidentsQ3.csv')
incident4 = pd.read_csv('Incidents/incidentsQ4.csv')

incident1 = pd.concat([incident1, incident2, incident3, incident4], ignore_index=True)
incident1 = incident1[['callstart','borough','ccgname','ccgcluster','lsoa','wardname','wardID']]
incident1['time'] = pd.to_datetime(incident1['callstart'])
incident1['time_Month'] = incident1['time'].map(lambda x: x.month)
incident1['time_Year'] = incident1['time'].map(lambda x: x.year)

incident1 = incident1[(incident1['time_Year'] == 2018)] #only take incident data in 2018

incident1.head()

In [ ]:
#Get the average number of incidents per borough per month
incident_borough = incident1.groupby(['borough','time_Month'],as_index=False).agg({"callstart": "count"})
incident_borough = incident_borough.groupby(['borough'],as_index=False).agg({"callstart": "mean"})

incident_borough.head()

In [ ]:
incident1['borough'].value_counts().plot(kind='bar')

In [ ]:
incident_borough.to_csv('incidents2018_borough_mean.csv') #export to scv to visualize in tableau